In [1]:
pip install -U python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

True

In [4]:
input_file = "20210311_v0.2" # no need for file extension, will be used later for file output
df = pd.read_csv("{}.csv".format(input_file))

### Dataset

The dataset used here seems to have some duplicates (see `unique` vs. `count`), and the dataset shows the same UEN with varying entity names - probably because company has been renamed, can confirm this through www.bizfile.gov.sg.


In [5]:
df.describe()

,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
count,24245,24245,24245,24245
unique,362,23231,23899,3
top,2020-09-16,202028206K,ARR SQUARED PTE. LTD.,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
freq,198,5,3,10638


In [6]:
df

,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
0,2020-01-01,53407676M,GRAVITY FILM,INFORMATION AND COMMUNICATIONS
1,2020-01-01,53407679C,SMARTMOUTH,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
2,2020-01-01,53407682C,INNOVIC TECHNOLOGY,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
3,2020-01-01,53407694K,SUPREM9 SOLUTIONS,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
4,2020-01-01,53407706D,THE LOVERS,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
...,...,...,...,...
24240,2020-12-17,T20VC0183A,SEAVI ADVENT EQUITY VII FUND VCC,FINANCIAL AND INSURANCE ACTIVITIES
24241,2020-12-23,T20VC0185D,WELLINGTON MANAGEMENT FUNDS (SINGAPORE) VCC,FINANCIAL AND INSURANCE ACTIVITIES
24242,2020-12-23,T20VC0187G,PENCO CAPITAL VCC,FINANCIAL AND INSURANCE ACTIVITIES
24243,2020-12-23,T20VC0190G,RAINMAKING VENTURES (S) VCC,FINANCIAL AND INSURANCE ACTIVITIES


So let's remove the duplicates, and only take the last item of entities with duplicate entries...

In [20]:
df = df.drop_duplicates(subset="Entity Profile UEN", keep="last")
df

,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
0,2020-01-01,53407676M,GRAVITY FILM,INFORMATION AND COMMUNICATIONS
1,2020-01-01,53407679C,SMARTMOUTH,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
2,2020-01-01,53407682C,INNOVIC TECHNOLOGY,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
3,2020-01-01,53407694K,SUPREM9 SOLUTIONS,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
4,2020-01-01,53407706D,THE LOVERS,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
...,...,...,...,...
24240,2020-12-17,T20VC0183A,SEAVI ADVENT EQUITY VII FUND VCC,FINANCIAL AND INSURANCE ACTIVITIES
24241,2020-12-23,T20VC0185D,WELLINGTON MANAGEMENT FUNDS (SINGAPORE) VCC,FINANCIAL AND INSURANCE ACTIVITIES
24242,2020-12-23,T20VC0187G,PENCO CAPITAL VCC,FINANCIAL AND INSURANCE ACTIVITIES
24243,2020-12-23,T20VC0190G,RAINMAKING VENTURES (S) VCC,FINANCIAL AND INSURANCE ACTIVITIES


In [21]:
df.to_csv("{}-filtered.csv".format(input_file), index=False) # re-write to new file, don't include dataframe's index

### Part 1

We first begin by crawling for the website addresses from existing data sources.

In [8]:
import requests
import bs4
import re
import time
import random
import os
from urllib.parse import urlparse, parse_qs
from dotenv import load_dotenv

In [9]:
load_dotenv()

PROXIES = os.getenv("PROXIES")

In [13]:
url = "https://httpbin.org/ip"

response = requests.get(url)
my_ip = response.json()['origin']

proxies = {}

proxies_string = PROXIES.split(",") # split our proxies into array e.g. ["proxy1.com", "proxy2.com"]

for i, p in enumerate(proxies_string):
    proxies[i] = {
        "http": p,
        "https": p
    }

proxy_count = len(proxies)

for i in range(proxy_count):
    try:
        response = requests.get(url, proxies=proxies[i])
        ip = response.json()['origin']
        print(ip)
        # if ip is not my_ip:
    except Exception as e:
        print("Proxy {} is down, error={}".format(i, e))

Proxy 0 is down, error=HTTPSConnectionPool(host='httpbin.org', port=443): Max retries exceeded with url: /ip (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 proxy authorization required')))
Proxy 1 is down, error=HTTPSConnectionPool(host='httpbin.org', port=443): Max retries exceeded with url: /ip (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 proxy authorization required')))
Proxy 2 is down, error=HTTPSConnectionPool(host='httpbin.org', port=443): Max retries exceeded with url: /ip (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 proxy authorization required')))
Proxy 3 is down, error=HTTPSConnectionPool(host='httpbin.org', port=443): Max retries exceeded with url: /ip (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 proxy authorization required')))
Proxy 4 is down, error=HTTPSConnectionPool(host='httpbin.org', port=443): Ma

KeyboardInterrupt: 

In [25]:
def get_random_user_agent():
    ua_strings = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"
        "Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1.2 Safari/605.1.15",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15",
        "Mozilla/5.0 (iPad; CPU OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 10; SM-A205U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 10; SM-N960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Mobile Safari/537.36"
    ]
 
    return random.choice(ua_strings)


'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'

In [ ]:
# Target URL to scrape
#

url = "https://www.google.com/search?q={}&sourceid=chrome&ie=UTF-8"
output_file = "{}-result.csv".format(input_file)
size = len(df) # Size of dataset

max_tries = 15

for i in range(0, size):
    entity = df.loc[i] # current row record
    entity_uen = entity[1] # entity UEN
    entity_name = entity[2] # entity name
    
    # Randomise search terms
    search_terms = [
        "{} singapore website".format(entity_name),
        "{} sg website".format(entity_name)
    ]
    
    search_terms_count = len(search_terms)
    
    # Prepare the CSV to write out
    formatted_csv_row = "\"{}\",\"{}\",\"{}\",\"{}\"".format(
        entity[0],
        entity[1],
        entity[2],
        entity[3]
    )

    # Accessing the webpage
    print("Searching for id={} {} {}".format(i, entity_uen, entity_name))
    
    for j in range(max_tries):
        try:
            url_new = url.format(entity_name)
            
            # Setup User Agent headers, attempt to imitate a "browser-like" request to the webpage
            headers = requests.utils.default_headers()
            headers.update({
                'User-Agent': get_random_user_agent()
            })

            # Pick a random rotating proxy
            id = random.randint(0, proxy_count)
            
            # Now, we query the target URL using a random proxy
            resp = requests.get(url_new, headers=headers, proxies=proxies[id])
            
            # Use BeautifulSoup to parse the HTML
            soup = bs4.BeautifulSoup(resp.text, "html.parser")
            
            # Now look for the specific elements
            headers = soup.find_all("h3")
            links = soup.find_all(href=re.compile(r'\/url\?q=')) # pick top 10 search results & its link
            
            for i in range(10):
                google_url = links[i].get("href").strip()
                parsed_url = parse_qs(google_url)
                formatted_csv_row += ",{}".format(parsed_url["/url?q"][0])
            
            # Finally we write out to file by appending
            f = open(output_file, "a")
            f.write(formatted_csv_row + "\n")
            f.close()
            
            print("    [SUCCESS] Wrote to file for id={} on retry={}".format(i, j))
    
            break # next for loop
        except Exception as e:
            # print("    [ERROR] Failed at id={} retry={} error={}, retrying...".format(i, j, e))
            
            if j == max_tries - 1:
                f = open("{}-failed.csv".format(input_file), "a")
                f.write(formatted_csv_row)
                f.close()
            continue # retry

    time.sleep(random.randint(1, 3)) # Randomise the waiting time


### Old Codes

The following code attempts to:

- crawl Google Search for the first 10 results
- then it grabs the URLs so that we can crawl them again for the metadata

In [ ]:

# f = open(output_file, "w")

# f.write("Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7,Link 8,Link 9,Link 10\n")

# for i in range(1, 20):
#     entity = df.loc[i] # current row record
#     entity_name = entity[2] # entity name
#     query = "{} singapore website".format(entity_name)
    
#     print("Searching for id={} \"{}\"".format(i, query))
    
#     # prepare to search "<entity name> singapore website" for more localised search context
#     req = requests.get(url.format(query))
#     soup = bs4.BeautifulSoup(req.text, "html.parser")
#     headers = soup.find_all("h3")
#     links = soup.find_all(href=re.compile(r'\/url\?q=')) # pick top 10 search results & its link
    
#     # file write here
    
#     formatted_csv_row = "{},{},{},{}".format(
#         entity[0],
#         entity[1],
#         entity[2],
#         entity[3]
#     )
    
#     for i in range(10):
#         google_url = links[i].get('href')
#         parsed_url = parse_qs(google_url)
#         formatted_csv_row += ",{}".format(parsed_url['/url?q'][0])
#     f.write(formatted_csv_row + "\n")
# f.close()


